## Vamos trabalhar com os dados abertos do ONS - Operador Nacional do Sistema Elétrico.

### Optei por trabalhar com os dados de Carga de Energia. Definição conforme site: [Dados de carga por subsistema numa data de referência em base diária.](https://dados.ons.org.br/dataset/carga-energia)


### Nessa primeira validação, faremos uma breve comparação entre a base bruta e a convertida em parquet.

In [1]:
import os
import pyspark

from pyspark import SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [2]:
sc = SparkContext.getOrCreate()

spark = (SparkSession
      .builder
      .getOrCreate()
     )

print('ApplicationID:', sc.applicationId)

ApplicationID: local-1630709313833


### Caminhos de Leitura e Load dos Dataframes

In [3]:
raw_path = '../data/raw/'
raw_path_list = [raw_path + x for x in os.listdir(raw_path)]
raw_path_list.sort()

dfs = [spark.read.csv(x, header= True, multiLine= True, sep= ";") for x in raw_path_list]

df_raw = dfs[0]
for i in range(1,len(dfs)):
    df_raw = df_raw.union(dfs[i])

In [4]:
bind_path = '../data/refined/01_bind/'

df_bind = spark.read.parquet(bind_path)

### Exploração dos Dataframes

In [5]:
for variavel in df_bind.columns:
    print(f'Variável \'{variavel}\':')
    print(f'Quantidade de registros nulos na base raw: {df_raw.filter(F.col(variavel).isNull()).count()}')
    print(f'Quantidade de registros nulos no bind: {df_bind.filter(F.col(variavel).isNull()).count()}')
    print(50 * '-')
    print(f'Quantidade de registros únicos na base raw: {df_raw.select(variavel).dropDuplicates().count()}')
    print(f'Quantidade de registros únicos no bind: {df_bind.select(variavel).dropDuplicates().count()}')
    print('\n')

Variável 'id_subsistema':
Quantidade de registros nulos na base raw: 0
Quantidade de registros nulos no bind: 0
--------------------------------------------------
Quantidade de registros únicos na base raw: 4
Quantidade de registros únicos no bind: 4


Variável 'nom_subsistema':
Quantidade de registros nulos na base raw: 0
Quantidade de registros nulos no bind: 0
--------------------------------------------------
Quantidade de registros únicos na base raw: 4
Quantidade de registros únicos no bind: 4


Variável 'din_instante':
Quantidade de registros nulos na base raw: 0
Quantidade de registros nulos no bind: 0
--------------------------------------------------
Quantidade de registros únicos na base raw: 7913
Quantidade de registros únicos no bind: 7913


Variável 'val_cargaenergiamwmed':
Quantidade de registros nulos na base raw: 17
Quantidade de registros nulos no bind: 17
--------------------------------------------------
Quantidade de registros únicos na base raw: 31625
Quantidade d

In [6]:
sc.stop()